In [105]:
import sys, os, random
import csv
import json
import numpy as np
from collections import defaultdict, OrderedDict

In [106]:
#temperature = 0.0
temperature = 1.0
#model = "gpt-4o-mini-2024-07-18"
model = "ft:gpt-4o-mini-2024-07-18:uedin:semantic-inequiv-bob-run-0-round-6-generation-0:AyS2gJFd"
max_tokens = 256
#n = 1
n = 9

In [107]:
#filename_base = "./gpt-4o-mini"
filename_base = "./sinq-gpt-4o-mini"

In [108]:
filename_to_openai = filename_base+"_to_openai.jsonl"
filename_from_openai = filename_base+"_from_openai.jsonl"

In [109]:
system_message = \
"""You are an expert computer scientist. Your task is to take a C/C++ function and determine whether it contains a security vulnerability.
Write your output the following JSON format:

```json
{
  "vulnerability": true or false
}
```

Do not write anything else.
"""

def user_message_fn_0(example):
    return f"""Function:

```
{example["func"]}
```
"""

In [110]:
def example_to_openai_api_format(example):
    messages = [{"role": "system", "content": system_message}, 
                {"role": "user", "content": user_message_fn_0(example)}]
    rv = {
        "model": model,
        "messages": messages,
        "temperature": temperature,
        "max_tokens": max_tokens,
        "n": n,
        "response_format": { "type": "json_object" }
    }
    return json.dumps(rv)

In [111]:
dataset = []

with open("./dataset/test.jsonl") as in_fs:
    for line in in_fs:
        dataset.append(json.loads(line))

In [112]:
with open(filename_to_openai, "w") as out_fs:
    for example in dataset:
        example["llm_request"] = example_to_openai_api_format(example)
        print(example["llm_request"], file=out_fs)

In [113]:
#

In [114]:
responses_from_openai = dict()
with open(filename_from_openai, "r") as in_fs:
    for line in in_fs:
        response_k, response_v = json.loads(line)
        response_k_str = json.dumps(response_k)
        responses_from_openai[response_k_str] = response_v

for example in dataset:
    if ("llm_request" in example) and (example["llm_request"] in responses_from_openai):
        example["llm_response"] = responses_from_openai[example["llm_request"]]
        example["llm_prediction"] = []
        for k, choice in enumerate(example["llm_response"]["choices"]):
            parsed = json.loads(choice["message"]["content"])
            example["llm_prediction"].append(parsed["vulnerability"])

In [115]:
num_examples = 0
num_correct = 0

for example in dataset:
    if "llm_request" in example:
        num_examples += 1
        majority_prediction = (np.array(example["llm_prediction"]).mean() >= 0.5)
        if example["target"] == majority_prediction:
            num_correct += 1

accuracy = float(num_correct) / num_examples
print(f"Model: {model}, n: {n}, temperature: {temperature}, evaluated examples: {num_examples}, num. correct: {num_correct}, accuracy: {accuracy}")

Model: ft:gpt-4o-mini-2024-07-18:uedin:semantic-inequiv-bob-run-0-round-6-generation-0:AyS2gJFd, n: 9, temperature: 1.0, evaluated examples: 2732, num. correct: 1531, accuracy: 0.5603953147877013


In [116]:
# Model: gpt-4o-mini-2024-07-18, n: 1, temperature: 0.0, evaluated examples: 2732, num. correct: 1509, accuracy: 0.5523426061493412
# Model: ft:gpt-4o-mini-2024-07-18:uedin:semantic-inequiv-bob-run-0-round-6-generation-0:AyS2gJFd, n: 1, temperature: 0.0, evaluated examples: 2732, num. correct: 1519, accuracy: 0.5560029282576867

# Model: gpt-4o-mini-2024-07-18, n: 9, temperature: 1.0, evaluated examples: 2732, num. correct: 1506, accuracy: 0.5512445095168375
# Model: ft:gpt-4o-mini-2024-07-18:uedin:semantic-inequiv-bob-run-0-round-6-generation-0:AyS2gJFd, n: 9, temperature: 1.0, evaluated examples: 2732, num. correct: 1531, accuracy: 0.5603953147877013